# Python Pandas: A sample analysis

In Python, the package "pandas" is a highly effective way to analyse numerical and analytical data. The best way to demonstrate this is by showing below an example, using a sample dataset for the Titanic.

First, we need to import the packages that will be used. Numpy is a package that pandas builds off of, so we will import both of them. Additionally, we will import seaborn, as this will provide us with the dataset.

In [322]:
import numpy as np
import pandas as pd
import seaborn as sns

Our tasks upon loading up the Titanic dataset are as follows:
- Clean the dataset: 
    - Where are the missing values, can they be filled/need to be dropped? - df.isnull(), df.fillna(), df.dropna()
    - Are there any duplicates? - df.duplicated(), df.drop_duplicates()
    - Do any columns need renaming? - df.rename()
    - Any other data quality concepts you can check?
- Check the descriptive statistics - df.describe()
- Find the average fare and age for each class - df.groupby(), mean()
- Add a new column showing the fare as a percentage of total
- How many survivors were female compared to male?
- What interesting insights can you draw?

We will go through these in order.

## Loading the dataset

A simple line of code can load the following:

In [323]:
sns.load_dataset('titanic')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


What resembles a table is called a "DataFrame" in pandas, so this is what it shall be referred to as. As indicated by the tasks though, it is clear that there is some cleaning of the data that needs to be done - from the DataFrame we can see already some "NaN" values ("Non-applicable Number"), which must be removed or adjusted in some other way.

## Missing values
We can first see what values are missing in the data by showing a sum of the values that are returning non-applicable values.

In [324]:
ndf = sns.load_dataset('titanic')
print(ndf.isna().sum())


survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


We can see that, while most of the columns are NaN-free, the result of the deck column is over three quarters of the values listed are NaN, so it is perhaps better left out of the overall DataFrame. There are other columns that have NaN, but these are less than a quarter of the amount of entries, so they should be dealt with differently. For now, let's drop the deck column, as dropping the rows that have this column as a value of NaN would wipe out most of what we have listed.

With the rest of the dropped values, we can just clear the rows which have them, as we will still have plenty of data to work with.

In [325]:
# Allows for the dropping of an entire column with the provided header (axis=1 indicates column)
ndf = ndf.drop("deck",axis=1)

# Drops a row if any of the values within it are non-applicable
ndf = ndf.dropna()
ndf.isna().sum()


survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

As we can now see by the summation of NaNs above, there are now no missing values present in the data.


## Duplicate data

To deal with duplicated data, we can first see what values are actually duplicated. Duplicates will be copies of lines within the dataset already, so removing them will not provide a lack of information and will instead improve the reliability of any calculations done with the data.

In [326]:
# Presents a sum over all columns of what is duplicated
print(ndf.duplicated().sum())


38


In [327]:
# Once we know how many duplicates there are, we can drop them
ndf = ndf.drop_duplicates()


## Renaming columns and unnecessary columns

Not only are we going to rename columns here, but also there is a definite need to clean up the DataFrame and remove some of the unneeded values. By this we mean stuff that is already assumed to be the case by another column. Take the columns "survived" and "alive" for example. Both are yes and no answers, just one is written with 0 and 1 (False and True) and the other with text. It is obvious that any value with "survived" as "0" is not going to have "alive" as "yes", so since we already know the information we need from this, we can drop one of these columns (we have chosen to remove "alive" for the sake of this example, but either one is fine to remove).

We also have column pairs "embarked from" & "embarked"; "who" & "sex" and "pclass" & "class". Again, only one of these is needed, so we will remove them too. For renaming, we have some considerations. We can modify some headers to be easier to understand and modify some others to just change the wording (not because they are hard to understand necessarily, but just to be a bit clearer in general).

In [328]:
# Drop the headers with the listed names
ndf = ndf.drop(["who","alive","pclass","embarked"], axis=1)

# Rename the headers on the left of every name/value pair listed with the value on the right
ndf = ndf.rename(columns= {"embark_town":"embarked from", "sibsp":"siblings and spouses aboard",
                          "parch":"parents and children aboard", "adult_male":"adult male"})

ndf

,survived,sex,age,siblings and spouses aboard,parents and children aboard,fare,class,adult male,embarked from,alone
0,0,male,22.0,1,0,7.2500,Third,True,Southampton,False
1,1,female,38.0,1,0,71.2833,First,False,Cherbourg,False
2,1,female,26.0,0,0,7.9250,Third,False,Southampton,True
3,1,female,35.0,1,0,53.1000,First,False,Southampton,False
4,0,male,35.0,0,0,8.0500,Third,True,Southampton,True
...,...,...,...,...,...,...,...,...,...,...
883,0,male,28.0,0,0,10.5000,Second,True,Southampton,True
885,0,female,39.0,0,5,29.1250,Third,False,Queenstown,False
887,1,female,19.0,0,0,30.0000,First,False,Southampton,True
889,1,male,26.0,0,0,30.0000,First,True,Cherbourg,True


## Descriptive statistics

There is a very simple input that we can use to immediately see numerical calculations in various columns. In relation to the example given so far:

```py
ndf.describe()
```

By typing this command, we immediately get a showing of the count, mean, standard deviation and min to max points (in quarters) listed for every numerical column.

In [334]:
ndf.describe()

,survived,age,siblings and spouses aboard,parents and children aboard,fare,% of fare total
count,674.000000,674.000000,674.000000,674.000000,674.000000,674.000000
mean,0.416914,29.752849,0.537092,0.455490,35.720580,0.148368
std,0.493415,14.693174,0.947016,0.871511,54.101153,0.224713
min,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,20.000000,0.000000,0.000000,8.050000,0.033436
50%,0.000000,28.000000,0.000000,0.000000,16.100000,0.066872
75%,1.000000,39.000000,1.000000,1.000000,34.584400,0.143649
max,1.000000,80.000000,5.000000,6.000000,512.329200,2.127996


We can see by this useful stats, such as the average number of siblings and spouses on board and the maximum fare for the Titanic - it also helps with considering potential problems with these calculations. For example, is it viable to include the ludicrous prices for fares in the mean if the values rise so drastically after the third quartile?

## Average fare and age

This is also another simple task. Using another DataFrame function used in pandas package, the DataFrame can be grouped into classes (observed=False is included as to indicate all categories should be shown. This doesn't matter as all of the categories have an input, so they can all be seen anyway).

In [335]:
ndf.groupby(by='class',observed=False)[['age','fare']].mean()

,age,fare
class,,
First,38.051758,88.489035
Second,29.859684,22.168223
Third,25.180150,13.377507


In [331]:
ndf['% of fare total'] = (ndf['fare'] / ndf['fare'].sum()) * 100
ndf

,survived,sex,age,siblings and spouses aboard,parents and children aboard,fare,class,adult male,embarked from,alone,% of fare total
0,0,male,22.0,1,0,7.2500,Third,True,Southampton,False,0.030113
1,1,female,38.0,1,0,71.2833,First,False,Cherbourg,False,0.296080
2,1,female,26.0,0,0,7.9250,Third,False,Southampton,True,0.032917
3,1,female,35.0,1,0,53.1000,First,False,Southampton,False,0.220555
4,0,male,35.0,0,0,8.0500,Third,True,Southampton,True,0.033436
...,...,...,...,...,...,...,...,...,...,...,...
883,0,male,28.0,0,0,10.5000,Second,True,Southampton,True,0.043612
885,0,female,39.0,0,5,29.1250,Third,False,Queenstown,False,0.120973
887,1,female,19.0,0,0,30.0000,First,False,Southampton,True,0.124607
889,1,male,26.0,0,0,30.0000,First,True,Cherbourg,True,0.124607


In [332]:
ndf['survived'][ndf['survived'] == 1].count()

ndf.groupby(['sex','survived']).describe()

age                                                        \
                 count       mean        std   min    25%   50%    75%   max   
sex    survived                                                                
female 0          64.0  25.046875  13.618591  2.00  16.75  24.5  33.25  57.0   
       1         189.0  28.652116  14.006161  0.75  19.00  28.0  36.00  63.0   
male   0         329.0  32.007599  14.375488  1.00  22.00  29.0  40.00  74.0   
       1          92.0  27.224674  16.587770  0.42  17.75  27.5  36.00  80.0   

                siblings and spouses aboard            ...      fare  \
                                      count      mean  ...       75%   
sex    survived                                        ...             
female 0                               64.0  0.968750  ...  27.90000   
       1                              189.0  0.539683  ...  77.95830   
male   0                              329.0  0.486322  ...  27.72080   
       1                               92.0  0.413043  ...  52.13855   

                          % of fare total                                \
                      max           count      mean       std       min   
sex    survived                                                           
female 0         151.5500            64.0  0.094585  0.105897  0.028037   
       1         512.3292           189.0  0.234065  0.283527  0.030010   
male   0         263.0000           329.0  0.100384  0.139814  0.000000   
       1         512.3292            92.0  0.181327  0.320063  0.000000   

                                                         
                      25%       50%       75%       max  
sex    survived                                          
female 0         0.040562  0.063852  0.115885  0.629474  
       1         0.053996  0.109031  0.323805  2.127996  
male   0         0.032623  0.043612  0.115140  1.092389  
       1         0.038589  0.109187  0.216561  2.127996  

[4 rows x 40 columns]